In [1]:
# Movie_recommendation_BERT 
 
#0) read_data from google drive

#1) TMDN_Dataset
 #1.1) get genres & overview from movies
 #1.2) tokenize data using BERT_Tokenizer
 #1.3) make input_form : [CLS] + genres + [SEP] + overview + [SEP]
 #1.4) make input be tensor

 #1.5) normalize label(make mean 1)
  #1.5.1) popularity
  #1.5.2) vote_average
  #1.5.3) vote_count
 #1.6) make input_target : popularity + vote_average * vote_count

#2) TMDM_Model
 #2.1) using Bert_base model
 #2.2) get last_hidden_states
 #2.3) get apool & mpool

#3) TMDM_Loss
 #3.1) using BCE_Loss

#4) TMDM_Train

#5) TMDM_Fit

#6) TMDM_embedding_matrix
 #6.1) get embedding_matrix that has information about movies

#7) TMDM_Recommendation
 #7.1) get_recommendation_function
 #7.2) get_embedding_function
  #7.2.1) prepare dataset & position_data
  #7.2.2) get_embedding : word_embedding + token_type_embedding + position_embedding 

In [2]:
########################################################################
########################################################################
from google.colab import drive
drive.mount("/content/gdrive", force_remount = True)
# 저는  코랩이랑 구글 드라이브를 써서 요부분은 지아님 환경에 맞게 바꿔주셔야 할것 같아요! 
########################################################################
########################################################################
!pip install transformers

Mounted at /content/gdrive
     |████████████████████████████████| 1.3MB 5.8MB/s 
     |████████████████████████████████| 2.9MB 39.6MB/s 
     |████████████████████████████████| 1.1MB 72.6MB/s 
     |████████████████████████████████| 890kB 60.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=8de84a7d6f4a7472d3b02c711adad41caf497944b2e863eba6baf7d511452af1
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [3]:
#0) read_data from google drive

import pandas as pd

########################################################################
########################################################################
# kaggle tmdm 데이터 불러오는 것도 지아님에 맞게 바꿔야할 것 같아요!
movies = pd.read_csv('/content/gdrive/My Drive/tmdb_5000_movies.csv')
########################################################################
########################################################################
#movies = movies.drop(2656).reset_index(drop = True)
#movies = movies.drop(4139).reset_index(drop = True)
#movies = movies.drop(4339).reset_index(drop = True)
#movies = movies.drop(4398).reset_index(drop = True)
#movies = movies.drop(4427).reset_index(drop = True)
title = []
for i in range(len(movies)):
  title.append(movies['title'][i])

action_movies = []
for j in range(len(movies)):
  if 'Action' in movies['genres'][j]:
    action_movies.append(j)

fantasy_movies = []
for j in range(len(movies)):
  if 'Fantasy' in movies['genres'][j]:
    fantasy_movies.append(j)

comedy_movies = []
for j in range(len(movies)):
  if 'Comedy' in movies['genres'][j]:
    comedy_movies.append(j)

horror_movies = []
for j in range(len(movies)):
  if 'Horror' in movies['genres'][j]:
    horror_movies.append(j)

romance_movies = []
for j in range(len(movies)):
  if 'Romance' in movies['genres'][j]:
    romance_movies.append(j)

In [5]:
#1) TMDN_Dataset
 #1.1) get genres & overview from movies
 #1.2) tokenize data using BERT_Tokenizer
 #1.3) make input_form : [CLS] + genres + [SEP] + overview + tagline + [SEP]
 #1.4) make token_type_ids
 #1.5) make attention_mask
 #1.6) make input be tensor
  #1.6.1) input_ids
  #1.6.2) token_type_ids
  #1.6.3) attention_mask
 #1.7) normalize label(make mean 1)
  #1.7.1) popularity
  #1.7.2) vote_average
  #1.7.3) vote_count
  #1.7.4) revenue
 #1.8) make input_target : popularity + vote_average * vote_count

device = 'cuda'

import math
import torch
import numpy as np
from transformers import BertTokenizer, BertModel, BertConfig

BERT_Tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

class TMDM_Dataset(torch.utils.data.Dataset):
  def __init__(self, movie_data, max_len, tokenizer):
    self.movie_data = movie_data
    self.max_len = max_len
    self.min_len = 10
    self.tokenizer = tokenizer
    self.encoded_cls = self.tokenizer.convert_tokens_to_ids('[CLS]')
    self.encoded_sep = self.tokenizer.convert_tokens_to_ids('[SEP]')
    self.encoded_pad = self.tokenizer.convert_tokens_to_ids('[PAD]')
    self.popularity_mean = 21.49230058817409
    self.vote_average_mean = 6.092171559442011
    self.vote_count_mean = 690.2179887570269
    self.target_threshold = 2.04

  def __len__(self):
    return len(self.movie_data)

  def __getitem__(self, index):

    #1.1) get genres & overview from movies
    # genres
    genres = self.movie_data.loc[index]['genres']
    if genres == '[]':
      genres = '[{"id": -1, "name": "None"}]'
    genres = self.get_genres(genres)
    genres = ' '.join(genres)
    
    # overview
    overview = self.movie_data.loc[index]['overview']

    # tagline
    tagline = self.movie_data.loc[index]['tagline']


    #1.2) tokenize data using BERT_Tokenizer
    # genres_token & genres_encoded
    if self.check_null(genres) == True:
      genres = 'None'
    genres = genres.lower()
    genres_token = self.tokenizer.tokenize(genres)
    genres_encoded = self.tokenizer.encode(genres_token)[1:-1]

    # overview_token & overview_encoded
    if self.check_null(overview) == True or len(overview) == 1:
      overview = 'None'
    overview = overview.lower()
    overview_token = self.tokenizer.tokenize(overview)
    overview_encoded = self.tokenizer.encode(overview_token)[1:-1]

    # tagline_token & tagline_encoded
    if self.check_null(tagline) == True:
      tagline = 'None'
    tagline = tagline.lower()
    tagline_token = self.tokenizer.tokenize(tagline)
    tagline_encoded = self.tokenizer.encode(tagline_token)[1:-1]

    # content_encoded = overview_encoded + tagline_encoded
    content_encoded = overview_encoded + tagline_encoded

    #1.3) make input_form : [CLS] + genres + [SEP] + content + [SEP]
    if len(genres_encoded) + len(content_encoded) >= (self.max_len - 3):
      content_encoded = content_encoded[:(self.max_len - 3 - len(genres))]
       
    input_ids = [self.encoded_cls] + genres_encoded + [self.encoded_sep] + \
    content_encoded + [self.encoded_sep]
    
    #1.4) make token_type_ids
    sent1_length = len(genres_encoded)
    sent2_length = len(content_encoded)
    token_type_ids = [0] * (sent1_length + 2) + [1] * (sent2_length + 1)


    #1.5) make attention_mask
    attention_mask = [1] * len(input_ids)

    #1.6) padding input to max_len & make tensor form
    # input_ids
    padding_length = self.max_len - len(input_ids)
    input_ids = self.padding(input_ids, self.encoded_pad, padding_length)
    input_ids = torch.tensor(input_ids)
    input_ids = input_ids.long()
    input_ids = input_ids.to(device)

    # token_type_ids
    token_type_ids = self.padding(token_type_ids, self.encoded_pad, padding_length)
    token_type_ids = torch.Tensor(token_type_ids)
    token_type_ids = token_type_ids.long()
    token_type_ids = token_type_ids.to(device)

    # attention_mask
    attention_mask = self.padding(attention_mask, self.encoded_pad, padding_length)
    attention_mask = torch.Tensor(attention_mask)
    attention_mask = attention_mask.long()
    attention_mask = attention_mask.to(device)


    #1.7) normalize label(make mean 1)
    #1.7.1) popularity
    popularity = self.movie_data.loc[index]['popularity']
    popularity = np.divide(popularity, self.popularity_mean)
    
    #1.7.2) vote_average
    vote_average = self.movie_data.loc[index]['vote_average']
    vote_average = np.divide(vote_average, self.vote_average_mean)

    #1.7.3) vote_count
    vote_count = self.movie_data.loc[index]['vote_count']
    vote_count = np.divide(vote_count, self.vote_count_mean)


    #1.8) make input_target : popularity + vote_average * vote_count
    input_target = popularity + vote_average * vote_count
    if input_target >= self.target_threshold:
      input_target = torch.Tensor([1])
    else:
      input_target = torch.Tensor([0])
    input_target = input_target.to(device)


    # dictionary
    dictionary = {}

    dictionary['input_ids'] = input_ids
    dictionary['token_type_ids'] = token_type_ids
    dictionary['attention_mask'] = attention_mask
    dictionary['input_target'] = input_target

    return dictionary
    
  def get_genres(self, data):
    genres = []
    for i in range(len(data)):
      if data[i] == ':' and data[i + 1] == ' ' and data[i + 2] == '"':
        j = i + 3
        while data[j] != '"':
          genres.append(data[j])
          j = j + 1
        genres.append('/')
 
    genres = ''.join(genres)
    genres = genres.split('/')[:-1]
    return genres

  def padding(self, input, value, length):
    return input + [value] * length

  def check_null(self, input):
    if isinstance(input, str) == True:
      boolean = pd.isnull(input)
  
    elif isinstance(input, float) == True:
      boolean = math.isnan(input)

    return boolean

In [10]:
data = TMDM_Dataset(movies, 80, BERT_Tokenizer)[4401]
data

{'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0'),
 'input_ids': tensor([ 101, 2895, 4038, 2671, 4349,  102, 3904, 3904,  102,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0], device='cuda:0'),
 'input_target': tensor([0.], device='cuda:0'),
 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0

In [6]:
#2) TMDM_Dataloader

from torch.utils.data import DataLoader

loader_dataset = TMDM_Dataset(movie_data = movies, 
                              max_len = 80,
                              tokenizer = BERT_Tokenizer)

train_dataloader = DataLoader(loader_dataset, 
                             batch_size = 16, 
                             shuffle = True, 
                             drop_last = True)

In [7]:
#2) TMDM_Model
 #2.1) using Bert_base model
 #2.2) get last_hidden_states
 #2.3) get apool & mpool

import torch.nn as nn

BERT_Model = BertModel.from_pretrained("bert-base-uncased")

class recommendation_model(nn.Module):
  def __init__(self):
    super(recommendation_model, self).__init__()
    self.model = BERT_Model
    self.hidden = 768 * 2
    self.out = 1
    self.linear = nn.Linear(self.hidden, self.out)
    self.dropout = nn.Dropout(0.5)
    self.activation = nn.Sigmoid()

  def forward(self, input_ids, token_type_ids, attention_mask):
    
    #2.2) get last_hidden_states
    last_hidden, _ = self.model(input_ids, token_type_ids, attention_mask)
    
    #2.3) get apool & mpool
    apool = torch.mean(last_hidden, dim = 1)
    mpool, _ = torch.max(last_hidden, dim = 1)
    concat = torch.cat((apool, mpool), dim = 1)

    logit = self.dropout(concat)
    logit = self.linear(logit)
    logit = self.activation(logit)

    return logit

  def get_cls(self, input_ids, token_type_ids, attention_mask):

    #2.4) get cls from trained model
    last_hidden, _ = self.model(input_ids, token_type_ids, attention_mask)
    cls_data = last_hidden[:, 0, :]
    return cls_data

TMDM_Model = recommendation_model().to(device) 

In [11]:
TMDM_Model(data['input_ids'].unsqueeze(0), 
           data['token_type_ids'].unsqueeze(0), 
           data['attention_mask'].unsqueeze(0))

tensor([[0.4574]], device='cuda:0', grad_fn=<SigmoidBackward>)

In [9]:
#3) TMDM_Loss
 #3.1) using BCE_Loss

def TMDM_Loss(logit, target):
  loss_function = nn.BCELoss()
  loss = loss_function(logit, target)

  return loss

In [12]:
#4) TMDM_Train

from tqdm import tqdm

def TMDM_Train(dataloader, model, loss_function, optimizer):

  model.train()
  book = tqdm(dataloader, total = len(dataloader))
  total_loss = 0.0

  for bi, dictionary in enumerate(book):
    input_ids = dictionary['input_ids']
    token_type_ids = dictionary['token_type_ids']
    attention_mask = dictionary['attention_mask']
    input_target = dictionary['input_target']

    input_ids = input_ids.to(device)
    token_type_ids = token_type_ids.to(device)
    attention_mask = attention_mask.to(device)

    model.zero_grad()
    logit = model(input_ids, token_type_ids, attention_mask)

    loss = loss_function(logit, input_target)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    total_loss += loss

  average_loss = total_loss / len(dataloader)
  print(" average_loss: {0:.2f}".format(average_loss))

In [13]:
#5) TMDM_Fit

def TMDM_Fit(train_function, model, loss_function, epoches, learning_rate):
  optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)
  for i in range(epoches):
    print(f"epoches:{i+1}")
    print('train')
    train_function(train_dataloader, model, loss_function, optimizer)
########################################################################
########################################################################
# 훈련된 bert 저장 하는 건데 저장할 위치 지아님에 맞게 바꿔주세요!
  torch.save(model, '/content/gdrive/My Drive/' + f'TMDM_Model:{i + 1}')
########################################################################
########################################################################

In [ ]:
stop

In [14]:
TMDM_Fit(TMDM_Train, TMDM_Model, TMDM_Loss, 8, 2e-5)

  0%|          | 0/300 [00:00<?, ?it/s]

epoches:1
train


  0%|          | 0/300 [00:00<?, ?it/s]

 average_loss: 0.59
epoches:2
train


  0%|          | 0/300 [00:00<?, ?it/s]

...

 average_loss: 0.05
epoches:7
train


  0%|          | 0/300 [00:00<?, ?it/s]

 average_loss: 0.03
epoches:8
train


100%|██████████| 300/300 [01:01<00:00,  4.90it/s]


 average_loss: 0.02


In [15]:
########################################################################
########################################################################
# 요놈은 훈련된 bert 불러오는 건데 그냥 처음부터 훈련하셔도 되고 
# 제가 올린 훈련된 bert 모델 저장하시고 쓰셔도 됩니다!
Trained_TMDM = torch.load('/content/gdrive/My Drive/' + f'TMDM_Model:{8}')
########################################################################
########################################################################

In [ ]:
Trained_TMDM

In [17]:
logits = Trained_TMDM(data['input_ids'].unsqueeze(0), 
                      data['token_type_ids'].unsqueeze(0), 
                      data['attention_mask'].unsqueeze(0))
logits

tensor([[0.0008]], device='cuda:0', grad_fn=<SigmoidBackward>)

In [18]:
#6) TMDM_embedding_matrix
 #6.1) get embedding_matrix that has information about movies

def get_embedding_matrix(data, dataset, max_len, tokenizer, trained_model):
  embedding_matrix = []
  dataset = dataset(data, max_len, tokenizer)
  position_data = torch.arange(max_len)
  position_data = position_data.to(device)
  for i in range(len(data)):
    input = dataset[i]
    word_embedding = trained_model.model.embeddings.word_embeddings(input['input_ids'])
    token_type_embedding = trained_model.model.embeddings.token_type_embeddings(input['token_type_ids'])
    position_embedding = trained_model.model.embeddings.position_embeddings(position_data)
    embedding = word_embedding + token_type_embedding + position_embedding
    embedding_matrix.append(embedding)
  embedding_matrix = torch.stack(embedding_matrix, dim = 0)
  embedding_matrix = embedding_matrix.to(device)

  return embedding_matrix

In [19]:
#7) TMDM_Recommendation
 #7.1) get_recommendation_function
 #7.2) get_embedding_function
  #7.2.1) prepare dataset & position_data
  #7.2.2) get_embedding : word_embedding + token_type_embedding + position_embedding 

class get_titles():

  #7.1) get_recommendation_function
  def get_recommendations(self, 
                          data,
                          dataset, 
                          trained_model, 
                          movie_set,
                          max_len, 
                          tokenizer, 
                          how_to_do,
                          movies_matrix,
                          want_num):
    
    if how_to_do == 'bert_embedding_layer':
      print('please type a movie you like, then we will propose ' + f'{want_num}' + ' movies similar')
      movie = input('>>>  ')
      index = movie_set.index(movie)
      input_embedding = self.get_embedding(data, 
                                           dataset, 
                                           trained_model, 
                                           index, 
                                           max_len, 
                                           tokenizer)
      input_matrix = torch.zeros([movies_matrix.shape[0], max_len, 768])
      input_matrix[:, :, :] = input_embedding
      input_matrix = input_matrix.to(device)

      matrix_mul = torch.mul(input_matrix, movies_matrix)
      matrix_mul = matrix_mul.sum(1)
      matrix_mul = matrix_mul.sum(1)

      wanted_indexes = []
      similarity_lists = matrix_mul.tolist()
      for i in range(1, (want_num + 1)):
        sorted_lists = sorted(similarity_lists)
        wanted_value = sorted_lists[-i]
        wanted_index = similarity_lists.index(wanted_value)
        wanted_indexes.append(wanted_index)

      wanted_movies = []
      for j in range(len(wanted_indexes)):
        title = data['title'][wanted_indexes[j]]
        wanted_movies.append(title)
    return wanted_movies[1:]

  #7.2) get_embedding_function
  def get_embedding(self, data, dataset, trained_model, index, max_len, tokenizer):

    # 7.2.1) prepare dataset & position_data
    dataset = dataset(data, max_len, tokenizer)[index]
    position_data = torch.arange(max_len)
    position_data = position_data.to(device)
  
    # 7.2.2) get_embedding : word_embedding + token_type_embedding + position_embedding 
    word_embedding = trained_model.model.embeddings.word_embeddings(dataset['input_ids'])
    token_type_embedding = trained_model.model.embeddings.token_type_embeddings(dataset['token_type_ids'])
    position_embedding = trained_model.model.embeddings.position_embeddings(position_data)
    embedding = word_embedding + position_embedding + token_type_embedding

    return embedding

TMDM_Recommendation = get_titles()

In [20]:
embedding_matrix = get_embedding_matrix(movies, TMDM_Dataset, 80, BERT_Tokenizer, Trained_TMDM)
embedding_matrix = embedding_matrix[romance_movies]


In [21]:
TMDM_Recommendation.get_recommendations(movies, TMDM_Dataset, Trained_TMDM, title, 80, BERT_Tokenizer, 'bert_embedding_layer', embedding_matrix, 30)

please type a movie you like, then we will propose 30 movies similar
>>>  Avatar


['Transformers: Revenge of the Fallen',
 'The Matrix Revolutions',
 'Dawn of the Planet of the Apes',
 'The Edge',
 'The Mummy Returns',
 "Charlotte's Web",
 'Scott Pilgrim vs. the World',
 'Ice Age: Dawn of the Dinosaurs',
 'Con Air',
 'How Do You Know',
 'John Carter',
 'Spirit: Stallion of the Cimarron',
 'The Finest Hours',
 'Night at the Museum: Battle of the Smithsonian',
 'A Beautiful Mind',
 'Fantasia 2000',
 'Fantastic 4: Rise of the Silver Surfer',
 'Ice Age: The Meltdown',
 'Cats & Dogs 2 : The Revenge of Kitty Galore',
 'Gangster Squad',
 'Agora',
 'Real Steel',
 'Elektra',
 'Les Misérables',
 'Star Trek: Insurrection',
 'American Gangster',
 'The Bourne Legacy',
 'The Peanuts Movie',
 'Cloudy with a Chance of Meatballs']